In [ ]:
# Imports
import math

In [ ]:
input_repr_dict = catalog.load('input_repr_dict')

### Unpack all the dictionary keys

In [ ]:
J = input_repr_dict["J"]
M = input_repr_dict["M"]
compat = input_repr_dict["compat"]
dur = input_repr_dict["dur"]
due = input_repr_dict["due"]
part_id = input_repr_dict["part_id"]

In [ ]:
J

In [ ]:
J_upd = [[i - 1 for i in sub_list] for sub_list in J]

In [ ]:
J_upd

Tasks of each job

In [ ]:
J[0]

List of machines M

In [ ]:
M

Compatability matrix for a taks of a job

In [ ]:
compat

Duration of a task of a job

In [ ]:
dur[0][0]

Due date (max completion time) for a job

In [ ]:
# Divide by 15 to indicate the number of 15 minute blocks that are required for the task
due = [math.ceil(due_date/15) for due_date in due]

In [ ]:
due[0]

Task to machines mapping:

In [ ]:
task_to_machines = {
  0: [1, 2, 3, 4, 5, 6],  # HAAS
  1: [7, 8, 9, 10],  # Inspection
  2: [11],  # First Wash
  3: [12, 13, 14],  # Manual Prep
  4: [7, 8, 9, 10],  # Inspection (again)
  5: [15],  # Final Wash
  6: [16, 17]  # Final inspection
}

In [ ]:
dur

In [ ]:
# Divided by fifteen to express the number of 15 minute blocks are required for the task
dur_rounded_up = [[math.ceil(x/15) for x in sub_list] for sub_list in dur]

In [ ]:
dur_rounded_up

In [ ]:
# Iterate over the sublists to round up the duration
# If there are only five tasks in the job, pad them with 0s
for sublist in dur_rounded_up:
    # If the sublist has a length of 5
    if len(sublist) == 5:
        # Insert two zeros before the last two entries
        sublist.insert(-2, 0)
        sublist.insert(-2, 0)

In [ ]:
dur_rounded_up

In [ ]:
task_to_machine_list = list(task_to_machines.values())

### Define the linear program

In [ ]:
# Example parameters
T = range(1, 337)  # Time horizon: 1 to 336
J = ['batch1', 'batch2']  # Example batch set
K = { 'batch1': [1, 2, 3], 'batch2': [1, 3, 2] }  # Example task sequences
M = { 1: ['machine1'], 2: ['machine2'], 3: ['machine3'] }  # Example machine sets
d = { ('batch1', 1): 2, ('batch1', 2): 3, ('batch1', 3): 2,
      ('batch2', 1): 4, ('batch2', 3): 1, ('batch2', 2): 3 }  # Example durations
f = { 'batch1': 100, 'batch2': 150 }  # Example promised delivery times

In [ ]:
import pulp

def linear_program(T, J, K, M, d, f):
    # Variables
    y = pulp.LpVariable.dicts("y", ((j, k, m, t) for j in J for k in K[j] for m in M[k] for t in T), 0, 1, pulp.LpBinary)
    x = pulp.LpVariable.dicts("x", ((j, k, m, t) for j in J for k in K[j] for m in M[k] for t in T), 0, 1, pulp.LpBinary)

    # Problem definition
    prob = pulp.LpProblem("SchedulingProblem", pulp.LpMaximize)
    
    # -- Objective -- 
    # Objective function: Minimize the starting times of tasks
    prob += pulp.lpSum(f[j] - (t * x[j, K[j][-1], m, t]) for j in J for m in M[K[j][-1]] for t in T)
                        
    # -- Constraints -- 
    # Each task in each batch starts once and only on one machine
    for j in J:
        for k in K[j]:
            prob += pulp.lpSum(y[j, k, m, t] for m in M[k] for t in T) == 1
     
    # A task only occupies a machine for the duration of processing time if the task is set to start on said machine
    for j in J:
        for k in K[j]:
            for m in M[k]:
                for t in T:
                    if t + d[j, k] - 1 <= max(T):
                        # Ensuring x values correspond correctly with the y start times
                        for t_prime in range(t, t + d[j, k]):
                            prob += x[j, k, m, t_prime] >= y[j, k, m, t_prime]

    # Total time the task occupies the machines equals the processing duration
    for j in J:
        for k in K[j]:
            for m in M[k]:
                prob += pulp.lpSum(x[j, k, m, t] for t in T) == d[j, k]
                
    # Only one machine can be used at a time
    for t in T:
        for k, m in M.items():
            mach = m[0]
            prob += pulp.lpSum(x[j, k, mach, t] for j in J) <= 1

    # Tasks must be done in the order specified by the batch
    for j in J:
        for idx in range(len(K[j]) - 1):
            k1 = K[j][idx]
            k2 = K[j][idx + 1]
            for m1 in M[k1]:
                for m2 in M[k2]:
                    prob += (pulp.lpSum((t + d[j, k1]) * y[j, k1, m1, t] for t in T) <= 
                             pulp.lpSum(t * y[j, k2, m2, t] for t in T))
                    
    # Tasks cannot be stopped midway, but have to be completed once they start
    for j in J:
        for k in K[j]:
            for m in M[k]:
                for t in T:
                    if t + d[j, k] - 1 <= max(T):
                        for t_prime in range(t, t + d[j, k]):
                            prob += x[j, k, m, t_prime] <= pulp.lpSum(y[j, k, m, t_prime_start] for t_prime_start in range(max(1, t_prime - d[j, k] + 1), t_prime + 1))
        
    # Solve the problem
    prob.solve()
    
    print("Status:", pulp.LpStatus[prob.status])
    
    # Print the optimal value of the objective function
    print("Optimal Value:", pulp.value(prob.objective))

    # Print the results
    for v in prob.variables():
        if v.varValue > 0.01:  #  and v.name.startswith('x')
            print(f"{v.name} = {v.varValue}")

            
    return x, y

# Example parameters
T = range(1, 337)  # Time horizon: 1 to 336
J = ['batch1', 'batch2']  # Example batch set
K = { 'batch1': [1, 2, 3], 'batch2': [1, 2, 3] }  # Example task sequences
M = { 1: ['machine1'], 2: ['machine2'], 3: ['machine3'] }  # Example machine sets
d = { ('batch1', 1): 2, ('batch1', 2): 3, ('batch1', 3): 4,
      ('batch2', 1): 2, ('batch2', 2): 3, ('batch2', 3): 4 }  # Example durations
f = { 'batch1': 100, 'batch2': 150 }  # Example promised delivery times

# Call the function
x, y = linear_program(T, J, K, M, d, f)
